#### In this notebook, Gold price, CRB price, Inflation Rate, M1 Money Supply and EURUSD data is scraped/downloaded, cleaned and saved to a pickle file to be used for EDA and modeling. 

In [1]:
import requests
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from datetime import date
import os

chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

requests.__path__

['/Users/yajasdwivedi/anaconda3/lib/python3.6/site-packages/requests']

## Scraping Gold Prices

In [2]:
driver = webdriver.Chrome(chromedriver)
driver.get('http://onlygold.com/Info/Search-Gold-Prices.asp')
time.sleep(1)

In [3]:
driver.refresh()

In [4]:
#Function that given date, extracts the table containing day and price.
def getGoldprice(date):
    #Find box where date needs to be entered
    search_box = driver.find_element_by_name("fmEndDate")
    #Type date in box
    search_box.send_keys(date)
    #Press ENTER
    search_box.send_keys(Keys.RETURN)
    #Store all information from page
    url = driver.page_source
    #Read HTML and extract desired table
    gold_price = pd.read_html(url,header=None,parse_dates=True)[12]
    return gold_price
    

In [5]:
#Function to generate list of dates - enter start and end date in format 'mm/dd/yy'
def dateGenerator(start,end):
    dates = pd.date_range(start, end, freq='20D')
    dates_list = []
    for date in dates:
        dates_list.append(date.strftime('%m/%d/%Y'))
    return dates_list

In [6]:
#Generate desired dates
dates = dateGenerator('08/02/16','04/20/18')
gold_prices = []
#Loop through list of dates and grab price data for each and store in a list
for date in dates:
    gold_prices.append(getGoldprice(date))

In [7]:
#Combine list of dataframes to make one dataframe
gold_prices_df = pd.concat(gold_prices)

In [8]:
#Clean up dataframe
gold_prices_df.reset_index(inplace = True)
gold_prices_df.drop_duplicates(inplace = True)
gold_prices_df.drop(('index'), axis = 1, inplace = True)
gold_prices_df.columns = ['DATE','Gold_Price']

In [9]:
gold_prices_df.DATE = pd.to_datetime(gold_prices_df.DATE)

In [10]:
#Remove $ symbol from prices...
gold_prices_with_dollars = gold_prices_df['Gold_Price']
def gross_text_to_float(gross_text):
    clean_text = gross_text.replace('$','').replace(',','')
    gross_as_float = float(clean_text)
    return gross_as_float
#...and convert to float
gold_price_column = []
for price in gold_prices_with_dollars:
    gold_price_column.append(gross_text_to_float(price))

In [11]:
gold_prices_df['Gold_Price_new'] = gold_price_column
gold_prices_df.drop('Gold_Price', axis = 1, inplace=True)
gold_prices_df.columns = ['DATE', 'Gold_Price']
gold_prices_df = gold_prices_df[gold_prices_df.Gold_Price != 0]
gold_prices_df.sort_values('DATE', ascending = True, inplace = True)

In [12]:
gold_prices_df.head()

,DATE,Gold_Price
14,2016-07-13,1342.75
13,2016-07-14,1323.60
12,2016-07-15,1327.00
11,2016-07-18,1334.70
10,2016-07-19,1330.90


In [13]:
gold_prices_df.to_pickle('/Users/yajasdwivedi/metis/LinearRegressionPriceGold/gold_prices_df.pkl')

## Preparing CRB Data

In [14]:
#CRB Data can be directly downloaded in csv format from 'https://quotes.wsj.com/index/XX/CRB/historical-prices' 
#by entering the desired dates

In [15]:
path_to_crb_file = '/Users/yajasdwivedi/metis/LinearRegressionPriceGold/CRB_HistoricalPrices.csv'
crb_df = pd.read_csv(path_to_crb_file)

In [16]:
crb_df.head()

,Date,Open,High,Low,Close
0,04/20/18,201.06,202.01,200.71,201.92
1,04/19/18,203.88,203.92,201.47,201.86
2,04/18/18,200.82,202.98,200.82,202.97
3,04/17/18,199.24,199.60,198.57,199.38
4,04/16/18,198.98,200.43,198.98,199.55


In [17]:
crb_df.drop([' Open', ' High', ' Low'], axis=1,inplace=True)
crb_df.columns = ['DATE','CRB_Price']

In [18]:
crb_df.DATE = pd.to_datetime(crb_df.DATE)

In [19]:
crb_df.head()

,DATE,CRB_Price
0,2018-04-20,201.92
1,2018-04-19,201.86
2,2018-04-18,202.97
3,2018-04-17,199.38
4,2018-04-16,199.55


In [20]:
crb_df.to_pickle('/Users/yajasdwivedi/metis/LinearRegressionPriceGold/crb_df.pkl')

## Scraping Inflation Data

In [21]:
driver.get('https://inflationdata.com/Inflation/Inflation_Rate/CurrentInflation.asp?reloaded=true')
time.sleep(1)

In [22]:
url = driver.page_source
inflation_rate = pd.read_html(url,header=0,parse_dates=True)[1]

In [23]:
inflation_rate.head()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Ave
0,2018,2.07%,2.21%,2.36%,2.46%,2.80%,2.87%,2.95%,2.70%,NaN,NaN,NaN,NaN,NaN
1,2017,2.50%,2.74%,2.38%,2.20%,1.87%,1.63%,1.73%,1.94%,2.23%,2.04%,2.20%,2.11%,2.13%
2,2016,1.37%,1.02%,0.85%,1.13%,1.02%,1.01%,0.84%,1.06%,1.46%,1.64%,1.69%,2.07%,1.26%
3,2015,-0.09%,-0.03%,-0.07%,-0.20%,-0.04%,0.12%,0.17%,0.20%,-0.04%,0.17%,0.50%,0.73%,0.12%
4,2014,1.58%,1.13%,1.51%,1.95%,2.13%,2.07%,1.99%,1.70%,1.66%,1.66%,1.32%,0.76%,1.62%


In [24]:
inflation_rate = inflation_rate.melt(id_vars=['Year'], var_name='month', value_name='inflation_rate')

In [25]:
inflation_rate.head()

,Year,month,inflation_rate
0,2018,Jan,2.07%
1,2017,Jan,2.50%
2,2016,Jan,1.37%
3,2015,Jan,-0.09%
4,2014,Jan,1.58%


In [26]:
#Create a new dataframe that contains all dates we are interested in - 'inflation_rate_df', note here that values for inflation
#are given monthly not daily - for the purposes of our model, we assign monthly values to every day we are interested in.
data_dates = pd.bdate_range(pd.to_datetime('13/07/2016', format='%d/%m/%Y'), periods=460).tolist()
inflation_rate_df = pd.DataFrame({"DATE": data_dates})
inflation_rate_df.sort_values('DATE', ascending = False, inplace = True)
inflation_rate_df.reset_index(inplace = True)
inflation_rate_df.drop('index',axis=1,inplace=True)
#inflation_rate_df.drop('level_0',axis=1,inplace=True)

In [27]:
inflation_rate['month_year'] = inflation_rate['month'] + ' ' + inflation_rate['Year']
inflation_rate = inflation_rate[inflation_rate.Year.isin(['2018','2017','2016']) & (inflation_rate.month != 'Ave')]
inflation_rate.month_year = pd.to_datetime(inflation_rate.month_year)

In [28]:
inflation_rate.set_index(inflation_rate['month_year'], inplace=True)
inflation_rate_df.set_index(inflation_rate_df['DATE'], inplace=True)
inflation_rate['per'] = inflation_rate.index.to_period('M')
inflation_rate_df['per'] = inflation_rate_df.index.to_period('M')

In [29]:
inflation_rate.head()

,Year,month,inflation_rate,month_year,per
month_year,,,,,
2018-01-01,2018,Jan,2.07%,2018-01-01,2018-01
2017-01-01,2017,Jan,2.50%,2017-01-01,2017-01
2016-01-01,2016,Jan,1.37%,2016-01-01,2016-01
2018-02-01,2018,Feb,2.21%,2018-02-01,2018-02
2017-02-01,2017,Feb,2.74%,2017-02-01,2017-02


In [30]:
#Merge inflation_rate and inflation_rate_df on the period column and drop columns we aren't interested in to get
#'inflation_rate_df_final'
inflation_rate.set_index(inflation_rate['per'], inplace=True)
inflation_rate_df.set_index(inflation_rate_df['per'], inplace = True)
inflation_rate_df_final = pd.merge(inflation_rate_df, inflation_rate, how="left", left_on=["per"], right_on=["per"])
inflation_rate_df_final.drop(['per','Year','month','month_year'], axis=1, inplace = True)

In [31]:
inflation_rate_df_final = inflation_rate_df_final.dropna(axis=0)

In [32]:
inflation_rate_as_object = inflation_rate_df_final['inflation_rate']
def gross_text_to_float_percent(gross_text):
    clean_text = gross_text.replace('%','').replace(',','')
    gross_as_float = float(clean_text)
    return gross_as_float
inflation_rate_float = []
for elem in inflation_rate_as_object:
    inflation_rate_float.append(gross_text_to_float_percent(elem))
inflation_rate_df_final['inflation_rate_percent'] = inflation_rate_float

In [33]:
inflation_rate_df_final.drop('inflation_rate', axis=1,inplace=True)

In [34]:
inflation_rate_df_final.head()

,DATE,inflation_rate_percent
0,2018-04-17,2.46
1,2018-04-16,2.46
2,2018-04-13,2.46
3,2018-04-12,2.46
4,2018-04-11,2.46


In [35]:
inflation_rate_df_final.to_pickle('/Users/yajasdwivedi/metis/LinearRegressionPriceGold/inflation_rate_df_final.pkl')

## Preparing M1 Money Supply Data

In [36]:
#M1 data can be directly downloaded in csv format from 'https://fred.stlouisfed.org/series/M1' 
#by entering the desired dates

In [37]:
m1_file = '/Users/yajasdwivedi/metis/LinearRegressionPriceGold/M1SL.csv'
m1_df = pd.read_csv(m1_file)

In [38]:
m1_df.head()

,DATE,M1SL
0,2016-08-01,3317.7
1,2016-09-01,3329.7
2,2016-10-01,3336.9
3,2016-11-01,3349.6
4,2016-12-01,3341.9


In [39]:
m1_dates_df = pd.DataFrame({"DATE": data_dates})
m1_dates_df.sort_values('DATE', ascending = False, inplace = True)
m1_dates_df.reset_index(inplace = True)
m1_dates_df.drop('index',axis=1,inplace=True)
m1_df.DATE = pd.to_datetime(m1_df.DATE)
m1_df.set_index(m1_df['DATE'], inplace=True)
m1_df['per'] = m1_df.index.to_period('M')
m1_dates_df.set_index(m1_dates_df['DATE'], inplace=True)
m1_dates_df['per'] = m1_dates_df.index.to_period('M')

In [40]:
m1_df.head()

,DATE,M1SL,per
DATE,,,
2016-08-01,2016-08-01,3317.7,2016-08
2016-09-01,2016-09-01,3329.7,2016-09
2016-10-01,2016-10-01,3336.9,2016-10
2016-11-01,2016-11-01,3349.6,2016-11
2016-12-01,2016-12-01,3341.9,2016-12


In [41]:
m1_dates_df.head()

,DATE,per
DATE,,
2018-04-17,2018-04-17,2018-04
2018-04-16,2018-04-16,2018-04
2018-04-13,2018-04-13,2018-04
2018-04-12,2018-04-12,2018-04
2018-04-11,2018-04-11,2018-04


In [42]:
m1_df_final = pd.merge(m1_df,m1_dates_df,how="right",left_on = ["per"],right_on=["per"])
m1_df_final.drop(['DATE_x','per'], axis=1, inplace=True)
columnsTitles=["DATE_y","M1SL"]
m1_df_final=m1_df_final.reindex(columns=columnsTitles)
m1_df_final.columns = ['DATE','M1SL']
m1_df_final.sort_values('DATE', ascending = False, inplace = True)
m1_df_final.reset_index(inplace = True)
m1_df_final.drop('index',axis=1,inplace=True)

In [43]:
m1_df_final = m1_df_final.dropna(axis=0)

In [44]:
m1_df_final.head()

,DATE,M1SL
12,2018-03-30,3663.2
13,2018-03-29,3663.2
14,2018-03-28,3663.2
15,2018-03-27,3663.2
16,2018-03-26,3663.2


In [45]:
m1_df_final.to_pickle('/Users/yajasdwivedi/metis/LinearRegressionPriceGold/m1_df_final.pkl')

## Preparing EURUSD Data

In [46]:
#EURUSD data can be directly downloaded in csv format from 'https://www.investing.com/currencies/eur-usd-historical-data' 
#by entering the desired dates

In [47]:
path_to_eurusd_file = '/Users/yajasdwivedi/metis/LinearRegressionPriceGold/EUR_USD Historical Data.csv'
eurusd_df = pd.read_csv(path_to_eurusd_file)
eurusd_df.Date = pd.to_datetime(eurusd_df.Date)
eurusd_df.head()

,Date,Price,Open,High,Low,Change %
0,2018-04-20,1.2288,1.2346,1.2354,1.2250,-0.47%
1,2018-04-19,1.2346,1.2373,1.2402,1.2329,-0.22%
2,2018-04-18,1.2373,1.2369,1.2398,1.2343,0.02%
3,2018-04-17,1.2370,1.2378,1.2413,1.2335,-0.08%
4,2018-04-16,1.2380,1.2330,1.2395,1.2325,0.40%


In [48]:
eurusd_df.drop(['Open','High','Low','Change %'],axis=1,inplace=True)
eurusd_df.columns = ['DATE','EURUSD']
eurusd_df.head()

,DATE,EURUSD
0,2018-04-20,1.2288
1,2018-04-19,1.2346
2,2018-04-18,1.2373
3,2018-04-17,1.2370
4,2018-04-16,1.2380


In [49]:
eurusd_df.to_pickle('/Users/yajasdwivedi/metis/LinearRegressionPriceGold/eurusd_df.pkl')